In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests

In [2]:
# Import the API key
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Nouadhibou,MR,2021-05-28 02:44:50,20.9310,-17.0347,64.38,82,0,17.27
1,1,Albany,US,2021-05-28 02:44:50,42.6001,-73.9662,55.98,62,99,5.10
2,2,Cherskiy,RU,2021-05-28 02:44:50,68.7500,161.3000,59.67,30,100,20.62
3,3,Port Lincoln,AU,2021-05-28 02:44:50,-34.7333,135.8667,61.77,57,97,9.64
4,4,Dunedin,NZ,2021-05-28 02:44:51,-45.8742,170.5036,47.80,83,100,8.99


In [5]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [16]:
# Get the maximum temperature
max_temp = city_data_df["Max Temp"]
temps = []

for temp in max_temp:
    temps.append(max(temp, 0))

In [18]:
# Heatmap of temperature
# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Heatmap of wind
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [24]:
# Filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
11,11,Atar,MR,2021-05-28 02:39:52,20.5169,-13.0499,83.28,21,6,7.11
12,12,San Patricio,US,2021-05-28 02:40:40,28.0170,-97.5169,81.32,90,56,13.65
14,14,Puerto Ayora,EC,2021-05-28 02:44:52,-0.7393,-90.3518,75.54,93,87,7.00
16,16,Avarua,CK,2021-05-28 02:44:53,-21.2078,-159.7750,82.45,83,75,13.80


In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
11,Atar,MR,83.28,20.5169,-13.0499,
12,San Patricio,US,81.32,28.0170,-97.5169,
14,Puerto Ayora,EC,75.54,-0.7393,-90.3518,
16,Avarua,CK,82.45,-21.2078,-159.7750,


In [29]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [33]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found...skipping.")

In [34]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
11,Atar,MR,83.28,20.5169,-13.0499,Oumou Elghoura
12,San Patricio,US,81.32,28.0170,-97.5169,"Motel 6 Sinton, TX"
14,Puerto Ayora,EC,75.54,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Avarua,CK,82.45,-21.2078,-159.7750,Paradise Inn


In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [39]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))